# Defining the field $\mathbb{F}_q$ and the polynomial ring $\mathbb{F}_q[T]$

Import libraries:

In [1]:
import os
import sys
import gzip
import lzma
import time
import pickle
import pprint
import datetime
import itertools
from functools import reduce

def printTemp(*args):
  print("\x1b[K", *args, end='\r')

Define base field by prime power:

In [17]:
q = 3

Calculate the field $F_q$:

($g$ is a generator of the multiplicative group of $F_q$)

In [3]:
Fq = GF(q,'g')
g = Fq.gen()

Define the polynomial rings $\mathbb{F}_q[T]$, $\mathbb{F}_q[T,X]$, and $\mathbb{F}_q[T][X]$:

In [4]:
FqT.<T> = Fq[]
FFqT.<T> = FunctionField(Fq)
FqTX.<T,X> = Fq[]
FFqT_X.<X> = FFqT[]
sep_TX = FqTX.hom([T,X],FFqT_X)

# Helper Functions

## Polynomial algebra and number theory

### Factors of a polynomial

In [5]:
def prime_factors(p):
    return set(pr for pr,n in list(p.factor()))
def cross_multiply(l):
    if len(l) == 0:
        return set(1)
    else:
        return set(p*q for p in l[0] for q in cross_multiply(l[1:]))
def all_factors(p):
    list_of_powers = [set(pr^i for i in range(n+1)) for pr,n in list(p.factor())]
    return cross_multiply(list_of_powers)
def is_monicList(l):
    for p in l:
        if p != 0:
            return p.is_monic()
    return False

### Euler $\varphi$ function

In [6]:
def qNorm(p):
    return q^p.degree()
def product(iterable):
    return reduce(operator.mul, iterable, 1)
def EulerPhi(n, p):
    pfs = prime_factors(p)
    pf_norms = [qNorm(pf) for pf in pfs]
    return qNorm(p)^n * product(1-pfn^(-n) for pfn in pf_norms)

### Extended gcd of a list of polynomials

In [7]:
def my_xgcd(l):
    if len(l) < 2:
        raise ValueError
    elif len(l) == 2:
        return l[0].xgcd(l[1])
    else: # len(l) > 2
        xgcd_rest = my_xgcd(l[1:])
        pgcd = l[0].xgcd(xgcd_rest[0])
        # + here is tuple concatenation:
        return pgcd[:2] + tuple(pgcd[2]*xgcd_rest[i] for i in range(1,len(l)))


### List of polynomials modulo a polynomial

In [8]:
@cached_function
def numsMod_deg(n):
    # output in FqT
    if n == 0:
        return [FqT.zero()]
    elif n > 0:
        return [f+FqT(T)*num for f in Fq for num in numsMod_deg(n-1)]
    else:
        raise ValueError
def numsMod(p):
    # output in FqT/p
    p = FqT(p)
    # modp = FqT.quotient(p, 'T')
    # return [modp(num) for num in numsMod_deg(p.degree())]
    return numsMod_deg(p.degree())

# numsMod(T^2+T)

### Lists of pairs modulo a polynomial

In [10]:
@cached_function
def pairsMod_deg(n):
    nums = numsMod_deg(n)
    return [(n1,n2) for n1 in nums for n2 in nums]
def pairsMod(p):
    p = FqT(p)
    return pairsMod_deg(p.degree())

def nonzeroPairs_deg(n):
    return [(n1,n2) for n1,n2 in pairsMod_deg(n) if (n1,n2) != (0,0)]
def nonzeroPairsMod(p):
    p = FqT(p)
    return nonzeroPairs_deg(p.degree())

@cached_function
def monicPairs_deg(n):
    return [pair for pair in nonzeroPairs_deg(n) if is_monicList(pair)]
def monicPairs(p):
    p = FqT(p)
    return monicPairs_deg(p.degree())

### Lists of cusps

In [10]:
@cached_function
def cusps(p):
    p = FqT(p)
    return [(p1,p2) for p1,p2 in monicPairs(p) if p.gcd(p1).gcd(p2) == 1]
    
@cached_function
def biCusps(p):
    p = FqT(p)
    cc = cusps(p)
    xgcd_coeffs = [my_xgcd(c+(p,))[1:] for c in cc]
    # return [(c,(-x[1],x[0])) for c,x in zip(cc,xgcd_coeffs)]
    return [((c[0],-x[1]),(c[1],x[0])) for c,x in zip(cc,xgcd_coeffs)]

# biCusps(T^2)

### List of pairs congruent to another pair modulo a divisor of a polynomial

In [11]:
# list of all pairs mod `p` which are congruent to the pair `r` modulo `m`,
# where `m` is a divisor of `p`
def pairsCongruentMod(r, m, p):
	m, p = FqT(m), FqT(p)
	if p % m != 0: raise ValueError
	n = FqT(p/m)
	return [(m*t1+r[0], m*t2+r[1]) for t1,t2 in pairsMod(n)]

# pairsCongruentMod((1,0), T, T^2+T)

## Reduction of list of Eisenstein pairs using known relations

In [12]:
def reducedTwoPairs(p, reduced=False):
	p = FqT(p)
	# list of (indices of) Eisenstein series of weight 1
	nnzPairs = nonzeroPairsMod(p)
	# list of products of two Eisenstein series of weight 1
	nnzTwoPairs = list(itertools.product(nnzPairs,repeat=2))

	# if we are not reducing the list of TwoPairs, return now
	if not reduced:
		return nnzTwoPairs
	
	# passing between products of two Eisenstein series and columns
	to_idx_dict = dict(zip(nnzTwoPairs,range(len(nnzTwoPairs))))
	to_twoPair_dict = dict(zip(range(len(nnzTwoPairs)),nnzTwoPairs))
	
	# constants
	zero, one = FqT.zero(), FqT.one()
	
	# dict-form of matrix of relations
	matrix_dict = {}
	r = 0
	def add_row(d): # `d` is a dict mapping twoPairs to matrix entries
		nonlocal r
		for k,v in d.items():
			matrix_dict[(r,to_idx_dict[k])] = v
		r += 1
	
	# scaling of left and right elements of product
	for l in Fq:
		if l != zero and l != one:
			for p1,p2 in nnzTwoPairs:
				lp1 = (l*p1[0], l*p1[1]) # l * p1
				add_row({(lp1,p2): l, (p1,p2): -one})
	# print(r+1, "scaling relations")
	
	# swapping of elements of products of distinct elements
	for p1,p2 in itertools.combinations(nnzPairs,2):
		add_row({(p1,p2): one, (p2,p1): -one})
	# print(r+1, "with swapping relations")
	
	# relations arising from additivity of e_Lambda(z)
	for pa,pb in nnzTwoPairs:
		pab = (pa[0]+pb[0], pa[1]+pb[1]) # pa + pb
		if pab == (zero, zero): continue

		if pa == pb:
			add_row({(pa,pb): one, (pab,pb): -one-one})
		else:
			add_row({(pa,pb): one, (pab,pb): -one, (pab,pa): -one})
	# print(r+1, "with adding relations")
	
	# linear relations arising from factorisation of `p`
	for m in prime_factors(p):
		n = FqT(p/m)
		for s in nonzeroPairsMod(n):
			for k in nnzPairs:
				d = {}
				for t in pairsCongruentMod(s, n, p):
					d[(t,k)] = one
			
				ms = (m*s[0],m*s[1]) # m * s
				if (ms,k) in d.keys():
					d[(ms,k)] = d[(ms,k)] - m
				else:
					d[(ms,k)] = -m
			
				add_row(d)
	# print(r+1, "with linear factor relations")
	
	# square quadratic relations arising from factorisation of `p`
	for m in prime_factors(p):
		n = FqT(p/m)
		for s in nonzeroPairsMod(n):
			d = {}
			for t in pairsCongruentMod(s, n, p):
				d[(t,t)] = one
			
			ms = (m*s[0],m*s[1]) # m * s
			if (ms,ms) in d.keys():
				d[(ms,ms)] = d[(ms,ms)] - m^2
			else:
				d[(ms,ms)] = -m^2
			
			add_row(d)

	# other quadratic relations arising from factorisation of `p`, for `q = 2`
	if q == 2:
		for m in prime_factors(p):
			n = FqT(p/m)
			for s in nonzeroPairsMod(n):
				d = {}
				for t1,t2 in itertools.combinations(pairsCongruentMod(s, n, p), 2):
					d[(t1,t2)] = one
				
				ms = (m*s[0],m*s[1]) # m * s
				for u in nonzeroPairsMod(m):
					nu = (n*u[0],n*u[1]) # n * u
					if (ms,nu) in d.keys():
						d[(ms,nu)] = d[(ms,nu)] - m
					else:
						d[(ms,nu)] = -m
				
				add_row(d)
	
	nrows, ncols = r+1, len(nnzTwoPairs)
	# print("Matrix of relations dimensions:", nrows, ncols)
	mat = matrix(FFqT, nrows, ncols, matrix_dict, sparse=True)
	# those which are not pivot columns of the echelonised form:
	# print("Calculating nonpivots:")
	non_pivots = mat.nonpivots()
	non_pivot_twoPairs = [to_twoPair_dict[col] for col in non_pivots]
	return non_pivot_twoPairs

# print(reducedTwoPairs(T^2,reduced=True))
# print(reducedTwoPairs(T^2,reduced=False))

## Drinfeld module functions

### Frobenius endomorphism and ring of skew polynomials:

In [13]:
frob = FqTX.hom([FqTX(T)^q,FqTX(X)^q])
frob
SkewTX.<tau> = FqTX['tau',frob]
# SkewTX
# phiaT = T +T*tau +tau^2; phiaT(T)

### $\varphi^{\overline{\pi} A}_p(x)$ for a polynomial $p \in \mathbb{F}_q[T]$, output in $\mathbb{F}_q(T)[X]$:

In [14]:
# the Carlitz thingy
phi_hom = FqT.hom([T+tau],SkewTX)

@cached_function
def phiA(p):
    return sep_TX(phi_hom(FqT(p)).operator_eval(X))

# (phiA(T^2+T)/phiA(T+1)/phiA(T)*phiA(1))(1/X).parent()

### Most primitive factor of $\varphi_p^{\overline{\pi} A}(x)$, output in $\mathbb{F}_q(T)[X]$:

In [15]:
@cached_function
def primPhiA(p):
    p = FqT(p)
    primes = list(prime_factors(p))
    return FFqT_X(phiA_inclExcl_div(p,primes))

def phiA_inclExcl_div(p,l):
    if len(l) == 0:
        return phiA(p)
    else:
        first, rest = l[0], l[1:]
        num = phiA_inclExcl_div(p,rest)
        denom = phiA_inclExcl_div(p/first,rest)
        return num / denom

# primPhiA(T^2+T) == phiA(T^2+T) / phiA(T+1) / phiA(T) * phiA(1)
# primPhiA(T^2+T).parent()
# str(primPhiA(T^3)).replace('X','e')

### $\overline{\pi} e_A \left(\frac{r}{N}\right) = e_{\overline{\pi} a} \left(\frac{\overline{\pi} r}{N}\right)$

In [16]:
@cached_function
def FqT_adj(N):
    # ext = FFqT_X.quotient(primPhiA(N), 'e1')
    why.<e> = FFqT[]
    poly = primPhiA(N)(e)
    ext.<e> = FFqT.extension(poly)
    return ext, e

# FqT_N, e = FqT_adj(T^2+T); FqT_N

In [17]:
@cached_function
def eA(r,N):
    ext, e1 = FqT_adj(N)
    return phiA(r % N)(e1)

# eA(T^3+T^2-T, T^2+T)
# eA(T^3+T^2-T, T^2+T).parent()

## Expansions of Eisenstein series

### Integrality factor

In [18]:
@cached_function
def int_factor(N):
    return FqT(N).radical()

# int_factor(T^2+T).parent()

### Expansion of an Eisenstein series at $\infty$

In [19]:
# X-expansion of an Eisenstein series at ∞ to order 'omax', exclusive
# 1 / (eA(r2,N) + phiA(r1,1/X))
@cached_function
def E2SeriesInfinity(r,N):
    r1,r2 = r
    r1 %= N; r2 %= N
    deg, lc = r1.degree(), r1.lc()

    radical = int_factor(N)
    omax = qNorm(N)

    if deg == -1: # r1 == 0 mod N
        return radical / eA(r2,N)
    else:
        frac = 1 -X^(q^deg) * (eA(r2,N)+phiA(r1)(1/X)) / lc
        # return from fraction field to polynomial ring
        fracfield = frac.parent()
        sect = fracfield.coerce_map_from(fracfield.ring()).section()
        frac = sect(frac)
        imax = omax - q^deg
        inexp = 0
        for i in range(imax+1):
            inexp += frac^i
        return radical * X^(q^deg) * inexp / lc +O(X^omax)

# E2SeriesInfinity((T,T+1), T^2), E2SeriesInfinity((T,1), T^2)
# E2SeriesInfinity((1,T), T^2).parent()

### Series expansion of a product of Eisenstein series at a cusp

In [20]:
@cached_function
def E2Series(r,biCusp,N):
    ((a,b),(c,d)) = biCusp
    R = [((a*r1+c*r2) % N, (b*r1+d*r2) % N) for r1,r2 in r]
    p = product(E2SeriesInfinity(rbc,N) for rbc in R) +O(X^qNorm(N))
    return p
    # return (R,p)

def E2Series_list(r,biCusp,N,deg):
    if deg > qNorm(N):
        print (deg, qNorm(N))
        raise ValueError(f"series expansion to {deg} > {qNorm(N)} terms requested`")

    series = E2Series(r,biCusp,N)
    l = series.list()
    if len(l) >= deg:
        return l[:deg]
    else: # len(l) < deg
        return l +[FqT.zero()]*(deg-len(l))

# E2Series([(1,0),(T,2),(2,T+1)], ((0,1),(-1,0)), T^2+T)
# E2Series_list(((1,0),(T,2),(2,T+1)), ((0,1),(-1,0)), T^2+T, 15)


### Coefficients of a product of Eisenstein series at all cusps, flattened and ordered by the exponent of the parameter at $\infty$

In [21]:
@cached_function
def E2SeriesAllCusps(r, N, cutoff):
    bcs = biCusps(N)
    # cutoff for the expansion at each cusp
    cusp_cut = ceil(cutoff/len(bcs))
    # dict of series expansions at each cusp
    all = {bc:E2Series_list(r,bc,N,cusp_cut) for bc in bcs}
    # flatten lists of coefficients and order by index/exponent
    out = [all[bc][i] for bc in bcs for i in range(cusp_cut)][:cutoff]
    return out

## Hypothetical list of linearly independent Eisenstein pairs

In [22]:
def hypoTwoPairs(p):
  p = FqT(p)
  nnzPairs = nonzeroPairsMod(p)
  # list of products of two Eisenstein series of weight 1
  twoPairs = list(itertools.product(nnzPairs,repeat=2))
  
  # constants
  zero, one = FqT.zero(), FqT.one()
  
  # pairs which together make a monic element of SL2(A/N)
  SL2Pairs = [(a,b) for a,b in twoPairs if (a[0]*b[1]-a[1]*b[0]).mod(p) == one
    and is_monicList(a) and (a[0].degree() < b[0].degree())]
  
  # (monic) duplicate pairs
  ApPairs = [(r,r) for r in pairsMod(p) if p.gcd(r[0]).gcd(r[1]) == one
    and is_monicList(r)]
  
  return SL2Pairs+ApPairs

# hypoTwoPairs(T^2)

# Calculating Rank

## Matrix of coefficients

In [23]:
def mat2PARI(m):
    mat = [] # a list of the rows
    for i in range(m.nrows()):
        row = [] # list of the elements of a row
        for j in range(m.ncols()):
            # convert element j in row i to str and append it to "row"
            row.append(str(m[i,j]))
        # use ", " between pairs of elements in "row"
        mat.append(', '.join(row))
    # add a new line character between each row in "mat" and return
    return '['+'; '.join(mat)+']'

def coeffMatrix(N, mode="full"):
    printTemp("Listing Eisenstein pairs")
    if mode == "full":
        redTwoPairs = reducedTwoPairs(N, False)
    elif mode == "slim":
        redTwoPairs = reducedTwoPairs(N, True)
    elif mode == "hypo":
        redTwoPairs = hypoTwoPairs(N)

    printTemp("Computing E2 expansions at ∞")
    cutoff = 2 * qNorm(N) * EulerPhi(2,N) / (q^2-1)
    # precompute Eisenstein expansions at infinity to avoid repeat calculations
    E2SeriesInfinity.precompute(
        set((r,N) for r in nonzeroPairsMod(N)),
        num_processes=os.cpu_count()
    )
    printTemp("Computing E2 expansions at cusps")
    E2SeriesAllCusps.precompute(
        set((r,N,cutoff) for r in redTwoPairs),
        num_processes=os.cpu_count()
    )
    printTemp("Computing final matrix of coeffs")
    mat = matrix([E2SeriesAllCusps(r, N, cutoff) for r in redTwoPairs])
    printTemp(f"Matrix has dimensions {mat.nrows()}×{mat.ncols()}")
    E2SeriesAllCusps.clear_cache()

    with gzip.open("matrices_"+mode+f"/q = {q}, N = {N}.txt.gz", 'wt') as txt:
        print(mat2PARI(mat), file=txt)
    with open(f"primPhiA/q = {q}, N = {N}.txt", 'w') as txt:
        print(str(primPhiA(N)).replace('X','e'), file=txt)

    return mat

# coeffMatrix(T^2)

## Matrix calculation and storage

In [24]:
# iterate over all polynomials of a requested degree
def matrix_iterate_deg(deg, mode="full"):
    Ns = [T^deg +pre_N for pre_N in numsMod_deg(deg)]
    for N in Ns:
        coeffMatrix(N, mode)
        print(f"Another one bites the dust: {N}")

In [26]:
matrix_iterate_deg(2,"hypo")

Another one bites the dust: T^212ffs
Another one bites the dust: T^2 + g*T
Another one bites the dust: T^2 + (g + 1)*T
Another one bites the dust: T^2 + Ts
Another one bites the dust: T^2 + gs
Another one bites the dust: T^2 + g*T + g
Another one bites the dust: T^2 + (g + 1)*T + g
Another one bites the dust: T^2 + T + g
Another one bites the dust: T^2 + (g + 1)
Another one bites the dust: T^2 + g*T + (g + 1)
Another one bites the dust: T^2 + (g + 1)*T + (g + 1)
Another one bites the dust: T^2 + T + (g + 1)
Another one bites the dust: T^2 + 1s
Another one bites the dust: T^2 + g*T + 1
Another one bites the dust: T^2 + (g + 1)*T + 1
Another one bites the dust: T^2 + T + 1


In [25]:
matrix_iterate_deg(3,"hypo")

Another one bites the dust: T^3 - T^2 - T - 1
Another one bites the dust: T^3 + T^2 - T - 1
Another one bites the dust: T^3 - T - 1ts
Another one bites the dust: T^3 - T^2 + T - 1
Another one bites the dust: T^3 + T^2 + T - 1
Another one bites the dust: T^3 + T - 1ts
Another one bites the dust: T^3 - T^2 - 1
Another one bites the dust: T^3 + T^2 - 1
Another one bites the dust: T^3 - 1cients
Another one bites the dust: T^3 - T^2 - T + 1
Another one bites the dust: T^3 + T^2 - T + 1
Another one bites the dust: T^3 - T + 1ts
Another one bites the dust: T^3 - T^2 + T + 1
Another one bites the dust: T^3 + T^2 + T + 1
Another one bites the dust: T^3 + T + 1ts
Another one bites the dust: T^3 - T^2 + 1
Another one bites the dust: T^3 + T^2 + 1
Another one bites the dust: T^3 + 1cients
Another one bites the dust: T^3 - T^2 - T
Another one bites the dust: T^3 + T^2 - T
Another one bites the dust: T^3 - Tcients
Another one bites the dust: T^3 - T^2 + T
Another one bites the dust: T^3 + T^2 + T
An

In [29]:
matrix_iterate_deg(4,"hypo")

Another one bites the dust: T^42048cients
Another one bites the dust: T^4 + T^3ents
Another one bites the dust: T^4 + T^2ents
Another one bites the dust: T^4 + T^3 + T^2
Another one bites the dust: T^4 + Tcients
Another one bites the dust: T^4 + T^3 + T
Another one bites the dust: T^4 + T^2 + T
Another one bites the dust: T^4 + T^3 + T^2 + T
Another one bites the dust: T^4 + 1cients
Another one bites the dust: T^4 + T^3 + 1
Another one bites the dust: T^4 + T^2 + 1
Another one bites the dust: T^4 + T^3 + T^2 + 1


: 

In [26]:
matrix_iterate_deg(5,"hypo")

: 

## Rank Calculation

In [ ]:
def stats(N):
    start = time.time()

    N = FqT(N)

    mat = coeffMatrix(N)
    
    rank2 = qNorm(N) * EulerPhi(2,N) / (q^2-1) + EulerPhi(2,N) / (q-1)
    printTemp(f"Rank of all weight 2 modular forms is {rank2}\n")

    printTemp(f"Calculating rank of {mat.nrows()}×{mat.ncols()} matrix")
    E2rank = mat.rank()
    printTemp(f"The final rank of the matrix is {E2rank}\n")

    n_nnz = sum(map(
        lambda row: sum(map(lambda entry: entry != 0, row)),
        mat
    ))
    n_mat = mat.nrows() * mat.ncols()

    end = time.time()

    return {
        "q": q,
        "N": N,
        "N, factored": N.factor(),
        "number of cusps": len(biCusps(N)),
        "final matrix of coefficients": mat,
        # "number of nonzero entries": n_nnz,
        # "fraction of nonzero entries": n_nnz / n_mat,
        "rank generated by Eisenstein series products": E2rank,
        "full rank of weight 2 Drinfeld modular forms": rank2,
        "time taken": str(datetime.timedelta(seconds=end-start))
    }

## Calling the rank function and saving its results

In [ ]:
@parallel(os.cpu_count())
def send_and_save(N):
    out = stats(N)

    # set working directory to script location
#     abspath = os.path.abspath(__file__)
#     dname = os.path.dirname(abspath)
#     os.chdir(dname)

    # save output to text file
    with open(f"results/q = {q}, N = {N}.txt", 'w') as file:
        pprint.pprint(out, width=10, stream=file)
    
    # pickle and compress output
    with lzma.open(f"results/q = {q}, N = {N}.xz", 'wb') as file:
        pickle.dump(out, file)

# iterate over all polynomials of a requested degree
def iterate_deg(deg):
    Ns = [T^deg +pre_N for pre_N in numsMod_deg(deg)]
    for results in send_and_save(Ns):
        print("Another one bites the dust")

    # for pre_N in numsMod_deg(deg):
    #     N = T^deg +pre_N
    #     send_and_save(N)

# get requested degree from command line and iterate
# Ndeg = Integer(sys.argv[2])
# iterate_deg(Ndeg)

## Rank Results

### $q = 2$, $\deg N = 2$

In [73]:
stats(T^2)

 Rank of all weight 2 modular forms is 28
 The final rank of the matrix is 28


{'q': 2,
 'N': T^2,
 'N, factored': T^2,
 'number of cusps': 12,
 'final matrix of coefficients': 44 x 32 dense matrix over Function field in e defined by e^2 + T*e + T,
 'rank generated by Eisenstein series products': 28,
 'full rank of weight 2 Drinfeld modular forms': 28,
 'time taken': '0:00:01.034445'}

In [116]:
stats(T^2+T)

 Rank of all weight 2 modular forms is 21
 The final rank of the matrix is 21


{'q': 2,
 'N': T^2 + T,
 'N, factored': T * (T + 1),
 'number of cusps': 9,
 'final matrix of coefficients': 21 x 24 dense matrix over Function field in e defined by e + 1,
 'rank generated by Eisenstein series products': 21,
 'full rank of weight 2 Drinfeld modular forms': 21,
 'time taken': '0:00:02.232486'}

### $q = 2$, $\deg N = 3$

In [74]:
stats(T^3)

 Rank of all weight 2 modular forms is 176
 Calculating rank of 560×256 matrix

KeyboardInterrupt: 

In [ ]:
stats(T^3+1)

 Rank of all weight 2 modular forms is 165
 Rank of matrix is 165


{'q': 2,
 'N': T^3 + 1,
 'N, factored': (T + 1) * (T^2 + T + 1),
 'number of cusps': 45,
 'finalMatrix': 425 x 240 dense matrix over Function field in e defined by e^3 + (T + 1)*e^2 + T*e + 1,
 'number of nonzero entries': 16429,
 'fraction of nonzero entries': 0.16106862745098038,
 'rank': 165,
 'full rank of weight 2 D-modular forms': 165,
 'time taken': '1:21:10.373216'}

In [ ]:
stats(T^3+T^2)

 Rank of all weight 2 modular forms is 132
 Rank of matrix is 132


{'q': 2,
 'N': T^3 + T^2,
 'N, factored': (T + 1) * T^2,
 'number of cusps': 36,
 'finalMatrix': 228 x 192 dense matrix over Function field in e defined by e^2 + T*e + 1,
 'number of nonzero entries': 7122,
 'fraction of nonzero entries': 0.16269188596491227,
 'rank': 132,
 'full rank of weight 2 D-modular forms': 132,
 'time taken': '1:04:37.713260'}

In [ ]:
stats(T^3+T^2+1)

 Rank of all weight 2 modular forms is 231
 Rank of matrix is 231


{'q': 2,
 'N': T^3 + T^2 + 1,
 'N, factored': T^3 + T^2 + 1,
 'number of cusps': 63,
 'finalMatrix': 1365 x 336 dense matrix over Function field in e defined by e^7 + (T^4 + T^2 + T + 1)*e^3 + (T^4 + T^3 + T)*e + T^3 + T^2 + 1,
 'number of nonzero entries': 72597,
 'fraction of nonzero entries': 0.15828754578754578,
 'rank': 231,
 'full rank of weight 2 D-modular forms': 231,
 'time taken': '18:06:13.080205'}

### $q = 2$, $\deg N = 4$

In [ ]:
stats(T*(T+1)*(T^2+T+1))

### $q = 3$, $\deg N = 2$

In [140]:
stats(T^2)

 Rank of all weight 2 modular forms is 117
 The final rank of the matrix is 117


{'q': 3,
 'N': T^2,
 'N, factored': T^2,
 'number of cusps': 36,
 'final matrix of coefficients': 279 x 162 dense matrix over Function field in e defined by e^6 + 2*T*e^4 + T^2*e^2 + T,
 'rank generated by Eisenstein series products': 117,
 'full rank of weight 2 Drinfeld modular forms': 117,
 'time taken': '0:26:43.345974'}

In [ ]:
stats(T^2+T)

 Forming set of Eisenstein pairs T

In [ ]:
stats(T^2+1)

 Rank of all weight 2 modular forms is 130
 Rank of matrix is 130


{'q': 3,
 'N': T^2 + 1,
 'N, factored': T^2 + 1,
 'number of cusps': 40,
 'finalMatrix': 430 x 180 dense matrix over Function field in e defined by e^8 + (T^3 + T)*e^2 + T^2 + 1,
 'number of nonzero entries': 11472,
 'fraction of nonzero entries': 0.14821705426356588,
 'rank': 130,
 'full rank of weight 2 D-modular forms': 130,
 'time taken': '0:59:11.282877'}

### $q = 4$, $\deg N = 2$

In [ ]:
stats(T^2)

In [ ]:
stats(T^2+g)

In [ ]:
stats(T^2+T)

In [ ]:
stats(T^2+T+g)

In [ ]:
stats(T^2+g*T)

In [ ]:
stats(T^2+g*T+g)

In [ ]:
stats(T^2+(g+1)*T)

In [ ]:
stats(T^2+(g+1)*T+g)

# Exotic Relations

In [ ]:
def exoticRelations(N):
    start = time.time()

    N = FqT(N)

    redTwoPairs = reducedTwoPairs(N)
    mat = coeffMatrix(N)
    
    printTemp("Finding exotic relations between Eisenstein series")
    rel_coeffs = mat.kernel().basis()
    # sort these relations by number of terms
    rel_coeffs = sorted(rel_coeffs, key=lambda vec:sum(c != 0 for c in vec))
    print(f"\rThere are {len(rel_coeffs)} exotic relations between Eisenstein series:")
    ind_R2P = {i:r for i,r in enumerate(redTwoPairs)}
    rel_strings = [
        f"{ri+1}: 0 = " + " + ".join(
            f"({c!s}) * E2[{ind_R2P[i][0]!s}] * E2[{ind_R2P[i][1]!s}]"
            for i,c in enumerate(coeff_list) if c != 0
        ) for ri, coeff_list in enumerate(rel_coeffs)
    ]
    for eqn in rel_strings: print(eqn)
    # rank of products of two Eisenstein series
    E2rank = mat.nrows() - len(rel_coeffs)

    end = time.time()


    return {
        "q": q,
        "N": N,
        "N, factored": N.factor(),
        "number of cusps": len(biCusps(N)),
        "finalMatrix": mat,
        "rank": E2rank,
        "time taken": str(datetime.timedelta(seconds=end-start))
    }

## $q = 2$, $\deg N = 2$

In [111]:
exoticRelations(T^2)

There are 16 exotic relations between Eisenstein series:
1: 0 = (1) * E2[(1, T + 1)] * E2[(0, 1)] + (e + T + 1) * E2[(1, T + 1)] * E2[(0, T + 1)] + (e + T + 1) * E2[(T + 1, 1)] * E2[(1, T)] + (1) * E2[(T + 1, 1)] * E2[(T + 1, T)] + (e + T + 1) * E2[(T + 1, T + 1)] * E2[(1, T + 1)] + (e + T + 1) * E2[(T + 1, T + 1)] * E2[(T + 1, 1)]
2: 0 = (1) * E2[(T + 1, 1)] * E2[(1, 0)] + (e + T + 1) * E2[(T + 1, 1)] * E2[(1, T)] + (e + T + 1) * E2[(T + 1, 1)] * E2[(T + 1, 0)] + (1) * E2[(T + 1, 1)] * E2[(T + 1, T)] + (e + T) * E2[(T + 1, T + 1)] * E2[(1, T + 1)] + (e + T) * E2[(T + 1, T + 1)] * E2[(T + 1, 1)]
3: 0 = (1) * E2[(T + 1, 1)] * E2[(1, 1)] + (1/T*e + 1) * E2[(T + 1, T + 1)] * E2[(1, 0)] + (1/T*e) * E2[(T + 1, T + 1)] * E2[(1, T)] + (1/T*e) * E2[(T + 1, T + 1)] * E2[(T + 1, 0)] + (1/T*e + 1) * E2[(T + 1, T + 1)] * E2[(T + 1, T)] + (1) * E2[(T + 1, T + 1)] * E2[(T + 1, 1)]
4: 0 = (1) * E2[(T + 1, T)] * E2[(1, 0)] + (1/T*e + (T + 1)/T) * E2[(T + 1, T + 1)] * E2[(1, 0)] + (1/T) * E2[(T + 1, T 

{'q': 2,
 'N': T^2,
 'N, factored': T^2,
 'number of cusps': 12,
 'finalMatrix': 44 x 32 dense matrix over Function field in e defined by e^2 + T*e + T,
 'rank': 28,
 'time taken': '0:00:01.850894'}

In [ ]:
exoticRelations(T^2+T)

0 relations:tic relations between products of two Eisenstein series


{'q': 2,
 'N': T^2 + T,
 'N, factored': T * (T + 1),
 'number of cusps': 9,
 'finalMatrix': 21 x 24 dense matrix over Function field in e defined by e + 1,
 'number of nonzero entries': 94,
 'fraction of nonzero entries': 0.1865079365079365,
 'rank': 21,
 'time taken': '0:00:01.267956'}

In [ ]:
exoticRelations(T^3)

384 relations:c relations between products of two Eisenstein series


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'q': 2,
 'N': T^3,
 'N, factored': T^3,
 'number of cusps': 48,
 'finalMatrix': 560 x 256 dense matrix over Function field in e defined by e^4 + (T^2 + T)*e^2 + T^2*e + T,
 'number of nonzero entries': 23139,
 'fraction of nonzero entries': 0.1614048549107143,
 'rank': 176,
 'time taken': '8:11:13.703207'}

In [ ]:
exoticRelations(T^3+1)

260 relations:c relations between products of two Eisenstein series


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'q': 2,
 'N': T^3 + 1,
 'N, factored': (T + 1) * (T^2 + T + 1),
 'number of cusps': 45,
 'finalMatrix': 425 x 240 dense matrix over Function field in e defined by e^3 + (T + 1)*e^2 + T*e + 1,
 'number of nonzero entries': 16429,
 'fraction of nonzero entries': 0.16106862745098038,
 'rank': 165,
 'time taken': '3:17:28.823961'}

In [ ]:
exoticRelations(T^3+T^2)


 Beginning calculation of statistics for T^3 + T^2
 Forming set of Eisenstein pairs

KeyboardInterrupt: 

In [ ]:
exoticRelations(T^3+T^2+1)

## $q = 3$

In [ ]:
exoticRelations(T^2)

162 relations:c relations between products of two Eisenstein series
1: 0 = (1) * E2[(2*T + 2, 2)] * E2[(2, 2*T + 2)] + (1) * E2[(2*T + 2, 2)] * E2[(2*T + 1, 2*T + 1)] + (1/T*e^4 + e^2) * E2[(2*T + 2, 2*T + 2)] * E2[(2, T + 1)] + (2/T*e^4 + 2*e^2 + 1) * E2[(2*T + 2, 2*T + 2)] * E2[(2, 2*T + 1)] + (1) * E2[(2*T + 2, 2*T + 2)] * E2[(2, 2*T + 2)] + (e^4 + (T + 1)*e^2) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T, T)] + (1/T*e^4 + e^2 + 2) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T + 1, 2)] + (2/T*e^4 + 2*e^2) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T + 1, T + 2)] + (1/T*e^4 + e^2) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T + 2, 1)] + (2) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T + 2, 2)] + (2/T*e^4 + 2*e^2 + 1) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T + 2, T + 1)]
2: 0 = (1) * E2[(2*T + 2, T + 2)] * E2[(2, T + 2)] + (1) * E2[(2*T + 2, T + 2)] * E2[(2*T + 1, 2*T + 1)] + (2/T*e^4 + 2*e^2 + 1) * E2[(2*T + 2, 2*T + 2)] * E2[(2, T)] + (1) * E2[(2*T + 2, 2*T + 2)] * E2[(2, T + 2)] + (1/T*e^4 + e^2) * E2[(2*T + 2, 2*T + 2)] * E2[(2, 2*T)] + 

{'q': 3,
 'N': T^2,
 'N, factored': T^2,
 'number of cusps': 36,
 'finalMatrix': 279 x 162 dense matrix over Function field in e defined by e^6 + 2*T*e^4 + T^2*e^2 + T,
 'number of nonzero entries': 6867,
 'fraction of nonzero entries': 0.151931501393867,
 'rank': 117,
 'time taken': '0:37:55.043845'}

In [ ]:
exoticRelations(T^2+T)

72 relations:ic relations between products of two Eisenstein series
1: 0 = (1) * E2[(2*T + 2, 2*T + 1)] * E2[(2*T + 2, 2)] + (2) * E2[(2*T + 2, 2*T + 1)] * E2[(2*T + 2, T)] + ((2/(T + 1))*e^2 + 1/(T + 1)) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T, 1)] + ((2/(T + 1))*e^2 + 2*T/(T + 1)) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T, 2)] + ((2/(T + 1))*e^2 + 1/(T + 1)) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T, T + 1)] + ((2/(T + 1))*e^2 + 2*T/(T + 1)) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T, T + 2)] + ((2/(T + 1))*e^2 + 1/(T + 1)) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T, 2*T + 1)] + ((2/(T + 1))*e^2 + 2*T/(T + 1)) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T, 2*T + 2)] + (2) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T + 2, T)] + (1) * E2[(2*T + 2, 2*T + 2)] * E2[(2*T + 2, 2*T + 1)]
2: 0 = (1) * E2[(2*T + 2, 2*T)] * E2[(2*T + 1, 1)] + (1) * E2[(2*T + 2, 2*T)] * E2[(2*T + 1, T + 1)] + (1) * E2[(2*T + 2, 2*T)] * E2[(2*T + 1, 2*T + 1)] + (1) * E2[(2*T + 2, 2*T)] * E2[(2*T + 2, 2)] + (1) * E2[(2*T + 2, 2*T)] * E2[(2*T + 2, T + 2)] + (2/T*e^2 + (

{'q': 3,
 'N': T^2 + T,
 'N, factored': T * (T + 1),
 'number of cusps': 32,
 'finalMatrix': 176 x 144 dense matrix over Function field in e defined by e^4 + (T + 2)*e^2 + 1,
 'number of nonzero entries': 3725,
 'fraction of nonzero entries': 0.14697758838383837,
 'rank': 104,
 'time taken': '0:47:27.045690'}

In [ ]:
exoticRelations(T^2+1)

300 relations:c relations between products of two Eisenstein series


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'q': 3,
 'N': T^2 + 1,
 'N, factored': T^2 + 1,
 'number of cusps': 40,
 'finalMatrix': 430 x 180 dense matrix over Function field in e defined by e^8 + (T^3 + T)*e^2 + T^2 + 1,
 'number of nonzero entries': 11472,
 'fraction of nonzero entries': 0.14821705426356588,
 'rank': 130,
 'time taken': '3:19:38.441768'}

# Hypothetical Eisenstein subset

### Hypothetical rank function

In [ ]:
def hypo_stats(N):
    start = time.time()

    N = FqT(N)
    printTemp("Beginning calculation of statistics for", N)

    printTemp("Forming set of Eisenstein pairs")
    redTwoPairs = hypoTwoPairs(N)
    
    printTemp("Calculating t-expansions of Eisenstein pairs")
    cutoff = 2 * qNorm(N) * EulerPhi(2,N) / (q^2-1)
    # precompute Eisenstein expansions at infinity to avoid repeat calculations
    E2SeriesInfinity.precompute(
        set((r,N) for r in nonzeroPairsMod(N)),
        num_processes=os.cpu_count()
    )
    cuts = [E2SeriesAllCusps(r, N, cutoff) for r in redTwoPairs]
    printTemp("Creating final matrix of coefficients")
    mat = matrix(cuts)
    r, c = mat.nrows(), mat.ncols()
    printTemp(f"Matrix has dimensions {r}×{c}")

    rank2 = qNorm(N) * EulerPhi(2,N) / (q^2-1) + EulerPhi(2,N) / (q-1)
    printTemp(f"Rank of all weight 2 modular forms is {rank2}\n")

    printTemp("Calculating rank of the matrix of coefficients,",
              f"with {mat.nrows()} rows and {mat.ncols()} columns")
    E2rank = mat.rank()
    printTemp(f"Rank of matrix is {E2rank}\n")

    n_nnz = sum(map(
        lambda row: sum(map(lambda entry: entry != 0, row)),
        mat
    ))
    n_mat = mat.nrows() * mat.ncols()

    end = time.time()


    return {
        "q": q,
        "N": N,
        "N, factored": N.factor(),
        "number of cusps": len(biCusps(N)),
        "final matrix of coefficients": mat,
        "number of nonzero entries": n_nnz,
        "fraction of nonzero entries": n_nnz / n_mat,
        "rank generated by hypothetical Eisenstein series products": E2rank,
        "number of hypothetical Eisenstein series products": len(redTwoPairs),
        "full rank of weight 2 Drinfeld modular forms": rank2,
        "time taken": str(datetime.timedelta(seconds=end-start))
    }

## Hypothetical Rank Results

### $q = 2$, $\deg N = 2$

In [ ]:
hypo_stats(T^2)

 Rank of all weight 2 modular forms is 28                                       
 Rank of matrix is 28                                                           


{'q': 2,
 'N': T^2,
 'N, factored': T^2,
 'number of cusps': 12,
 'final matrix of coefficients': 28 x 32 dense matrix over Function field in e defined by e^2 + T*e + T,
 'number of nonzero entries': 149,
 'fraction of nonzero entries': 0.16629464285714285,
 'rank generated by hypothetical Eisenstein series products': 28,
 'number of hypothetical Eisenstein series products': 28,
 'full rank of weight 2 Drinfeld modular forms': 28,
 'time taken': '0:00:23.741744'}

In [ ]:
hypo_stats(T^2+T)

 Rank of all weight 2 modular forms is 21                                       
 Rank of matrix is 21                                                           


{'q': 2,
 'N': T^2 + T,
 'N, factored': T * (T + 1),
 'number of cusps': 9,
 'final matrix of coefficients': 21 x 24 dense matrix over Function field in e defined by e + 1,
 'number of nonzero entries': 85,
 'fraction of nonzero entries': 0.16865079365079366,
 'rank generated by hypothetical Eisenstein series products': 21,
 'number of hypothetical Eisenstein series products': 21,
 'full rank of weight 2 Drinfeld modular forms': 21,
 'time taken': '0:00:24.267246'}

### $q = 2$, $\deg N = 3$

In [ ]:
hypo_stats(T^3)

 Calculating t-expansions of Eisenstein pairs                                   

 Rank of all weight 2 modular forms is 176                                      
 Rank of matrix is 176                                                          


{'q': 2,
 'N': T^3,
 'N, factored': T^3,
 'number of cusps': 48,
 'final matrix of coefficients': 176 x 256 dense matrix over Function field in e defined by e^4 + (T^2 + T)*e^2 + T^2*e + T,
 'number of nonzero entries': 6878,
 'fraction of nonzero entries': 0.15265447443181818,
 'rank generated by hypothetical Eisenstein series products': 176,
 'number of hypothetical Eisenstein series products': 176,
 'full rank of weight 2 Drinfeld modular forms': 176,
 'time taken': '0:19:44.660508'}

In [ ]:
hypo_stats(T^3+1)

 Rank of all weight 2 modular forms is 165                                      
 Rank of matrix is 165                                                          


{'q': 2,
 'N': T^3 + 1,
 'N, factored': (T + 1) * (T^2 + T + 1),
 'number of cusps': 45,
 'final matrix of coefficients': 165 x 240 dense matrix over Function field in e defined by e^3 + (T + 1)*e^2 + T*e + 1,
 'number of nonzero entries': 6246,
 'fraction of nonzero entries': 0.15772727272727272,
 'rank generated by hypothetical Eisenstein series products': 165,
 'number of hypothetical Eisenstein series products': 165,
 'full rank of weight 2 Drinfeld modular forms': 165,
 'time taken': '0:14:40.735335'}

In [ ]:
hypo_stats(T^3+T^2)

 Rank of all weight 2 modular forms is 132                                      
 Rank of matrix is 132                                                          


{'q': 2,
 'N': T^3 + T^2,
 'N, factored': (T + 1) * T^2,
 'number of cusps': 36,
 'final matrix of coefficients': 132 x 192 dense matrix over Function field in e defined by e^2 + T*e + 1,
 'number of nonzero entries': 4045,
 'fraction of nonzero entries': 0.15960385101010102,
 'rank generated by hypothetical Eisenstein series products': 132,
 'number of hypothetical Eisenstein series products': 132,
 'full rank of weight 2 Drinfeld modular forms': 132,
 'time taken': '0:03:35.596921'}

In [ ]:
hypo_stats(T^3+T^2+1)

 Rank of all weight 2 modular forms is 231                                      
 Rank of matrix is 231                                                          


{'q': 2,
 'N': T^3 + T^2 + 1,
 'N, factored': T^3 + T^2 + 1,
 'number of cusps': 63,
 'final matrix of coefficients': 231 x 336 dense matrix over Function field in e defined by e^7 + (T^4 + T^2 + T + 1)*e^3 + (T^4 + T^3 + T)*e + T^3 + T^2 + 1,
 'number of nonzero entries': 11487,
 'fraction of nonzero entries': 0.1479978354978355,
 'rank generated by hypothetical Eisenstein series products': 231,
 'number of hypothetical Eisenstein series products': 231,
 'full rank of weight 2 Drinfeld modular forms': 231,
 'time taken': '3:27:42.003567'}

### $q = 2$, $\deg N = 4$

In [44]:
hypo_stats(T*(T+1)*(T^2+T+1))

 Rank of all weight 2 modular forms is 855                                      
 Rank of matrix is 855                                                          


{'q': 2,
 'N': T^4 + T,
 'N, factored': T * (T + 1) * (T^2 + T + 1),
 'number of cusps': 135,
 'final matrix of coefficients': 855 x 1440 dense matrix over Function field in e defined by e^3 + e^2 + (T^2 + T)*e + 1,
 'number of nonzero entries': 176698,
 'fraction of nonzero entries': 0.14351689408706952,
 'rank generated by hypothetical Eisenstein series products': 855,
 'number of hypothetical Eisenstein series products': 855,
 'full rank of weight 2 Drinfeld modular forms': 855,
 'time taken': '5 days, 20:27:07.493311'}

In [ ]:
hypo_stats((T^2+T+1)^2)

In [24]:
hypo_stats(T^2*(T+1)^2)

 Rank of all weight 2 modular forms is 912                                      
 Rank of matrix is 912                                                          


{'q': 2,
 'N': T^4 + T^2,
 'N, factored': T^2 * (T + 1)^2,
 'number of cusps': 144,
 'final matrix of coefficients': 912 x 1536 dense matrix over Function field in e defined by e^4 + (T^2 + T + 1)*e^2 + (T^2 + T)*e + 1,
 'number of nonzero entries': 197365,
 'fraction of nonzero entries': 0.14089127033077486,
 'rank generated by hypothetical Eisenstein series products': 912,
 'number of hypothetical Eisenstein series products': 912,
 'full rank of weight 2 Drinfeld modular forms': 912,
 'time taken': '14 days, 4:22:33.719790'}

In [44]:
hypo_stats(T^3*(T+1))

 Rank of all weight 2 modular forms is 912                                      
 Rank of matrix is 912                                                          


{'q': 2,
 'N': T^4 + T^3,
 'N, factored': (T + 1) * T^3,
 'number of cusps': 144,
 'final matrix of coefficients': 912 x 1536 dense matrix over Function field in e defined by e^4 + (T^2 + T)*e^2 + T^2*e + 1,
 'number of nonzero entries': 197351,
 'fraction of nonzero entries': 0.14088127627010233,
 'rank generated by hypothetical Eisenstein series products': 912,
 'number of hypothetical Eisenstein series products': 912,
 'full rank of weight 2 Drinfeld modular forms': 912,
 'time taken': '14 days, 6:38:21.870465'}

In [45]:
hypo_stats(T*(T+1)^3)

 Rank of all weight 2 modular forms is 912                                      
 Rank of matrix is 912                                                          


{'q': 2,
 'N': T^4 + T^3 + T^2 + T,
 'N, factored': T * (T + 1)^3,
 'number of cusps': 144,
 'final matrix of coefficients': 912 x 1536 dense matrix over Function field in e defined by e^4 + (T^2 + T)*e^2 + (T^2 + 1)*e + 1,
 'number of nonzero entries': 197357,
 'fraction of nonzero entries': 0.140885559438962,
 'rank generated by hypothetical Eisenstein series products': 912,
 'number of hypothetical Eisenstein series products': 912,
 'full rank of weight 2 Drinfeld modular forms': 912,
 'time taken': '15 days, 19:34:43.761576'}

### $q = 3$, $\deg N = 2$

In [26]:
hypo_stats(T^2)

 Rank of all weight 2 modular forms is 117                                      
 Rank of matrix is 117                                                          


{'q': 3,
 'N': T^2,
 'N, factored': T^2,
 'number of cusps': 36,
 'final matrix of coefficients': 117 x 162 dense matrix over Function field in e defined by e^6 + 2*T*e^4 + T^2*e^2 + T,
 'number of nonzero entries': 2712,
 'fraction of nonzero entries': 0.1430832541943653,
 'rank generated by hypothetical Eisenstein series products': 117,
 'number of hypothetical Eisenstein series products': 117,
 'full rank of weight 2 Drinfeld modular forms': 117,
 'time taken': '0:08:36.827131'}

In [27]:
hypo_stats(T^2+T)

 Rank of all weight 2 modular forms is 104                                      
 Rank of matrix is 104                                                          


{'q': 3,
 'N': T^2 + T,
 'N, factored': T * (T + 1),
 'number of cusps': 32,
 'final matrix of coefficients': 104 x 144 dense matrix over Function field in e defined by e^4 + (T + 2)*e^2 + 1,
 'number of nonzero entries': 2259,
 'fraction of nonzero entries': 0.15084134615384615,
 'rank generated by hypothetical Eisenstein series products': 104,
 'number of hypothetical Eisenstein series products': 104,
 'full rank of weight 2 Drinfeld modular forms': 104,
 'time taken': '0:04:44.150380'}

In [28]:
hypo_stats(T^2+1)

 Rank of all weight 2 modular forms is 130                                      
 Rank of matrix is 130                                                          


{'q': 3,
 'N': T^2 + 1,
 'N, factored': T^2 + 1,
 'number of cusps': 40,
 'final matrix of coefficients': 130 x 180 dense matrix over Function field in e defined by e^8 + (T^3 + T)*e^2 + T^2 + 1,
 'number of nonzero entries': 3248,
 'fraction of nonzero entries': 0.13880341880341882,
 'rank generated by hypothetical Eisenstein series products': 130,
 'number of hypothetical Eisenstein series products': 130,
 'full rank of weight 2 Drinfeld modular forms': 130,
 'time taken': '0:30:15.821656'}

## Calling the hypo-rank function and saving its results

In [ ]:
@parallel(os.cpu_count())
def send_and_save(N):
    out = hypo_stats(N)

    # set working directory to script location
#     abspath = os.path.abspath(__file__)
#     dname = os.path.dirname(abspath)
#     os.chdir(dname)

    # save output to text file
    with open(f"hypo_results/q = {q}, N = {N}.txt", 'w') as file:
        pprint.pprint(out, width=10, stream=file)
    
    # pickle and compress output
    with lzma.open(f"hypo_results/q = {q}, N = {N}.xz", 'wb') as file:
        pickle.dump(out, file)

# iterate over all polynomials of a requested degree
def iterate_deg(deg):
    Ns = [T^deg +pre_N for pre_N in numsMod_deg(deg)]
    for results in send_and_save(Ns):
        print("Another one bites the dust")

    # for pre_N in numsMod_deg(deg):
    #     N = T^deg +pre_N
    #     send_and_save(N)

# get requested degree from command line and iterate
# Ndeg = Integer(sys.argv[2])
Ndeg = Integer(2)
iterate_deg(Ndeg)

# Rough Work

In [28]:
@parallel(4)
def f(n): return n*n

list(f([1,2,3]))

[(((1,), {}), 1), (((2,), {}), 4), (((3,), {}), 9)]